## 自己构建的网络

In [1]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
from keras.layers import Lambda
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input as resnet50_pre
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input as inceptionV3_pre
from keras.applications.xception import Xception
from keras.applications.xception import preprocess_input as xception_pre
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input as vgg16_pre
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input as vgg19_pre
from keras.preprocessing import image   
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils.np_utils import to_categorical
from tqdm import tqdm
from PIL import ImageFile  
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense, Input, Activation
from keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint  
import numpy as np
import pandas as pd
from glob import glob
import cv2                
import matplotlib.pyplot as plt    
import matplotlib.image as mpimg
%matplotlib inline 
import random
import os
import shutil
import h5py
import common
import pickle

Using TensorFlow backend.


## 对图片进行分类

In [2]:
common.divide_images()

In [3]:
small_models_and_features = {}
small_models_and_test_features = {}

In [4]:
def batch_extract_small_features():
    
    # VGG16
    base_model = VGG16(weights='imagenet', include_top=False, pooling='avg')
    features_name = common.extract_small_features(base_model, (224, 224), vgg16_pre)
    small_models_and_features['vgg16'] = features_name
    
    # VGG19
    base_model = VGG19(weights='imagenet', include_top=False, pooling='avg')
    features_name = common.extract_small_features(base_model, (224, 224), vgg19_pre)
    small_models_and_features['vgg19'] = features_name
    
    # ResNet50
    base_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')
    features_name = common.extract_small_features(base_model, (224, 224), resnet50_pre)
    small_models_and_features['resnet50'] = features_name
    
    # InceptionV3
    base_model = InceptionV3(weights='imagenet', include_top=False, pooling='avg')
    features_name = common.extract_small_features(base_model, (299, 299), inceptionV3_pre)
    small_models_and_features['inceptionV3'] = features_name
    
    # Xception
    base_model = Xception(weights='imagenet', include_top=False, pooling='avg')
    features_name = common.extract_small_features(base_model, (299, 299), xception_pre)
    small_models_and_features['xception'] = features_name

In [5]:
def batch_extract_small_test_features():
    
    # VGG16
    base_model = VGG16(weights='imagenet', include_top=False, pooling='avg')
    features_name = common.extract_small_test_features(base_model, (224, 224), vgg16_pre)
    small_models_and_test_features['vgg16'] = features_name
    
    # VGG19
    base_model = VGG19(weights='imagenet', include_top=False, pooling='avg')
    features_name = common.extract_small_test_features(base_model, (224, 224), vgg19_pre)
    small_models_and_test_features['vgg19'] = features_name
    
    # ResNet50
    base_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')
    features_name = common.extract_small_test_features(base_model, (224, 224), resnet50_pre)
    small_models_and_test_features['resnet50'] = features_name
    
    # InceptionV3
    base_model = InceptionV3(weights='imagenet', include_top=False, pooling='avg')
    features_name = common.extract_small_test_features(base_model, (299, 299), inceptionV3_pre)
    small_models_and_test_features['inceptionV3'] = features_name
    
    # Xception
    base_model = Xception(weights='imagenet', include_top=False, pooling='avg')
    features_name = common.extract_small_test_features(base_model, (299, 299), xception_pre)
    small_models_and_test_features['xception'] = features_name

In [7]:
batch_extract_small_features()

Found 1600 images belonging to 2 classes.
Found 400 images belonging to 2 classes.
20/20 [==============================] - ETA: 10:4 - ETA: 9:4 - ETA: 9: - ETA: 8: - ETA: 7: - ETA: 7: - ETA: 6: - ETA: 6: - ETA: 5: - ETA: 5: - ETA: 4: - ETA: 4: - ETA: 3: - ETA: 3: - ETA: 2: - ETA: 2: - ETA: 1: - ETA: 1: - ETA: 30s - 606s 30s/step
Found 1600 images belonging to 2 classes.
Found 400 images belonging to 2 classes.
20/20 [==============================] - ETA: 12:0 - ETA: 11:3 - ETA: 10:4 - ETA: 10:1 - ETA: 9:4 - ETA: 9: - ETA: 8: - ETA: 7: - ETA: 7: - ETA: 6: - ETA: 5: - ETA: 5: - ETA: 4: - ETA: 3: - ETA: 3: - ETA: 2: - ETA: 1: - ETA: 1: - ETA: 38s - 780s 39s/step
Found 1600 images belonging to 2 classes.
Found 400 images belonging to 2 classes.
20/20 [==============================] - ETA: 7: - ETA: 6: - ETA: 6: - ETA: 5: - ETA: 5: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 42s - ETA: 21 - 428s 21s/step
F

In [6]:
batch_extract_small_test_features()

Found 500 images belonging to 1 classes.
25/25 [==============================] - ETA: 8: - ETA: 9: - ETA: 9: - ETA: 11:1 - ETA: 12:1 - ETA: 12:4 - ETA: 12:5 - ETA: 12:1 - ETA: 12:0 - ETA: 11:3 - ETA: 10:5 - ETA: 10:1 - ETA: 9:2 - ETA: 8: - ETA: 7: - ETA: 7: - ETA: 6: - ETA: 5: - ETA: 4: - ETA: 3: - ETA: 3: - ETA: 2: - ETA: 1: - ETA: 44s - 1103s 44s/step
Found 500 images belonging to 1 classes.
25/25 [==============================] - ETA: 20:1 - ETA: 18:3 - ETA: 17:4 - ETA: 16:5 - ETA: 16:0 - ETA: 15:0 - ETA: 14:1 - ETA: 13:2 - ETA: 12:3 - ETA: 11:5 - ETA: 11:0 - ETA: 10:1 - ETA: 9:2 - ETA: 8: - ETA: 7: - ETA: 7: - ETA: 6: - ETA: 5: - ETA: 4: - ETA: 3: - ETA: 3: - ETA: 2: - ETA: 1: - ETA: 46s - 1177s 47s/step


W0410 18:46:55.649117  8116 deprecation.py:506] From C:\ProgramData\Anaconda3\envs\graduate\lib\site-packages\tensorflow\python\training\moving_averages.py:210: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Found 500 images belonging to 1 classes.
25/25 [==============================] - ETA: 10:5 - ETA: 10:4 - ETA: 9:5 - ETA: 9: - ETA: 8: - ETA: 8: - ETA: 7: - ETA: 7: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 5: - ETA: 5: - ETA: 4: - ETA: 4: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 2: - ETA: 2: - ETA: 1: - ETA: 1: - ETA: 51s - ETA: 25 - 647s 26s/step
Found 500 images belonging to 1 classes.
25/25 [==============================] - ETA: 15:2 - ETA: 12:3 - ETA: 11:1 - ETA: 10:0 - ETA: 9:3 - ETA: 8: - ETA: 8: - ETA: 7: - ETA: 7: - ETA: 6: - ETA: 6: - ETA: 5: - ETA: 5: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 3: - ETA: 3: - ETA: 2: - ETA: 2: - ETA: 1: - ETA: 1: - ETA: 53s - ETA: 26 - 667s 27s/step
Found 500 images belonging to 1 classes.
25/25 [==============================] - ETA: 22:4 - ETA: 21:4 - ETA: 20:2 - ETA: 19:0 - ETA: 18:2 - ETA: 17:4 - ETA: 16:4 - ETA: 15:3 - ETA: 14:4 - ETA: 13:4 - ETA: 12:5 - ETA: 12:0 - ETA: 11:0 - ETA: 10:0 - ETA: 9:1 - ETA: 8: - ETA: 7: - ETA: 6: - ETA: 5: - ETA: 4: - E

In [6]:
if len(small_models_and_features) == 0:
    small_models_and_features['vgg16'] = 'small_vgg16_features.npz'
    small_models_and_features['vgg19'] = 'small_vgg19_features.npz'
    small_models_and_features['resnet50'] = 'small_resnet50_features.npz'
    small_models_and_features['inceptionV3'] = 'small_inception_v3_features.npz'
    small_models_and_features['xception'] = 'small_xception_features.npz'

In [7]:
if len(small_models_and_test_features) == 0:
    small_models_and_test_features['vgg16'] = 'test_vgg16_features.npz'
    small_models_and_test_features['vgg19'] = 'test_vgg19_features.npz'
    small_models_and_test_features['resnet50'] = 'test_resnet50_features.npz'
    small_models_and_test_features['inceptionV3'] = 'test_inception_v3_features.npz'
    small_models_and_test_features['xception'] = 'test_xception_features.npz'

In [8]:
def model_train(key, dropout, optimizer, file_header):
    features = np.load(small_models_and_features[key])
    features_test = np.load(small_models_and_test_features[key])
    train_features = features['train']
    train_labels = features['train_label']
    valid_features = features['valid']
    valid_labels = features['valid_label']
    test_features = features_test['test']
    test_filenames = features_test['test_filename']
    
    model = Sequential()
    model.add(Dense(2048, input_shape=(train_features.shape[1],), activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(2048, activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    
    checkpointer = ModelCheckpoint(filepath='{0}.hdf5'.format(file_header), verbose=1, save_best_only=True)
    train_result = model.fit(train_features, train_labels, epochs=10, batch_size=common.batch_size,
          validation_data=(valid_features, valid_labels), verbose=1, callbacks=[checkpointer])
    
    with open('{0}_history.pkl'.format(file_header), 'wb') as f:
        pickle.dump(train_result.history, f)
    model.load_weights('{0}.hdf5'.format(file_header))
    prediction = model.predict(test_features, batch_size=common.batch_size)
    prediction = prediction[:, 0].clip(0.01, 0.99)
    test_fileindex = np.array([os.path.splitext(os.path.split(filename)[1])[0] for filename in test_filenames])
    data = np.stack([test_fileindex, prediction], axis=1)
    tmp = pd.DataFrame(data, columns=['id', 'label'])
    tmp['id'] = tmp['id'].apply(pd.to_numeric)
    submit_frame = pd.read_csv('dogs-vs-cats/sample_submission.csv')
    result = pd.merge(submit_frame, tmp, on="id", how='left')
    result = result.rename(index=str, columns={"label_y": "label"})
    result.dropna(axis=0, subset=['label'], inplace=True)
    result[['id','label']].to_csv('{0}_predict.csv'.format(file_header),index=False)

In [16]:
def plot_info(model_name, acc_lengend, loss_lengend):
    adam_3 = 'small_{0}_Adam_0.3_history.pkl'.format(model_name)
    adam_5 = 'small_{0}_Adam_0.5_history.pkl'.format(model_name)
    adam_7 = 'small_{0}_Adam_0.7_history.pkl'.format(model_name)
    sgd_3 = 'small_{0}_SGD_0.3_history.pkl'.format(model_name)
    sgd_5 = 'small_{0}_SGD_0.5_history.pkl'.format(model_name)
    sgd_7 = 'small_{0}_SGD_0.7_history.pkl'.format(model_name)
    with open(adam_3, 'rb') as file:
        history = pickle.load(file)
        acc_adam_3 = history['acc']
        loss_adam_3 = history['loss']
        val_acc_adam_3 = history['val_acc']
        val_loss_adam_3 = history['val_loss']
    
    with open(adam_5, 'rb') as file:
        history = pickle.load(file)
        acc_adam_5 = history['acc']
        loss_adam_5 = history['loss']
        val_acc_adam_5 = history['val_acc']
        val_loss_adam_5 = history['val_loss']
        
    with open(adam_7, 'rb') as file:
        history = pickle.load(file)
        acc_adam_7 = history['acc']
        loss_adam_7 = history['loss']
        val_acc_adam_7 = history['val_acc']
        val_loss_adam_7 = history['val_loss']
        
    with open(sgd_3, 'rb') as file:
        history = pickle.load(file)
        acc_sgd_3 = history['acc']
        loss_sgd_3 = history['loss']
        val_acc_sgd_3 = history['val_acc']
        val_loss_sgd_3 = history['val_loss']
        
    with open(sgd_5, 'rb') as file:
        history = pickle.load(file)
        acc_sgd_5 = history['acc']
        loss_sgd_5 = history['loss']
        val_acc_sgd_5 = history['val_acc']
        val_loss_sgd_5 = history['val_loss']
        
    with open(sgd_7, 'rb') as file:
        history = pickle.load(file)
        acc_sgd_7 = history['acc']
        loss_sgd_7 = history['loss']
        val_acc_sgd_7 = history['val_acc']
        val_loss_sgd_7 = history['val_loss']
    plt.figure(figsize=(16, 16))
    plt.subplot(431)
    plt.plot(range(1, len(acc_adam_3) + 1), acc_adam_3, label='train adam .3')
    plt.plot(range(1, len(val_acc_adam_3) + 1), val_acc_adam_3, label='valid adam .3')
    plt.legend(loc=acc_lengend, shadow=True, fontsize='x-large')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    
    plt.subplot(432)
    plt.plot(range(1, len(acc_adam_5) + 1), acc_adam_5, label='train adam .5')
    plt.plot(range(1, len(val_acc_adam_5) + 1), val_acc_adam_5, label='valid adam .5')
    plt.legend(loc=acc_lengend, shadow=True, fontsize='x-large')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    
    plt.subplot(433)
    plt.plot(range(1, len(acc_adam_7) + 1), acc_adam_7, label='train adam .7')
    plt.plot(range(1, len(val_acc_adam_7) + 1), val_acc_adam_7, label='valid adam .7')
    plt.legend(loc=acc_lengend, shadow=True, fontsize='x-large')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    
    plt.subplot(434)
    plt.plot(range(1, len(acc_sgd_3) + 1), acc_sgd_3, label='train sgd .3')
    plt.plot(range(1, len(val_acc_sgd_3) + 1), val_acc_adam_3, label='valid sgd .3')
    plt.legend(loc=acc_lengend, shadow=True, fontsize='x-large')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    
    plt.subplot(435)
    plt.plot(range(1, len(acc_sgd_5) + 1), acc_sgd_5, label='train sgd .5')
    plt.plot(range(1, len(val_acc_sgd_5) + 1), val_acc_sgd_5, label='valid sgd .5')
    plt.legend(loc=acc_lengend, shadow=True, fontsize='x-large')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    
    plt.subplot(436)
    plt.plot(range(1, len(acc_sgd_7) + 1), acc_sgd_7, label='train sgd .7')
    plt.plot(range(1, len(val_acc_sgd_7) + 1), val_acc_sgd_7, label='valid sgd .7')
    plt.xticks(range(1, len(acc_adam_3) + 1));
    plt.legend(loc=acc_lengend, shadow=True, fontsize='x-large')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    
    plt.subplot(437)
    plt.plot(range(1, len(loss_adam_3) + 1), loss_adam_3, label='train adam .3')
    plt.plot(range(1, len(val_loss_adam_3) + 1), val_loss_adam_3, label='valid adam .3')
    plt.legend(loc=acc_lengend, shadow=True, fontsize='x-large')
    plt.xlabel('Epochs')
    plt.ylabel('LogLoss')
    plt.subplot(438)
    plt.plot(range(1, len(loss_adam_5) + 1), loss_adam_5, label='train adam .5')
    plt.plot(range(1, len(val_loss_adam_5) + 1), val_loss_adam_5, label='valid adam .5')
    plt.legend(loc=loss_lengend, shadow=True, fontsize='x-large')
    plt.xlabel('Epochs')
    plt.ylabel('LogLoss');
    
    plt.subplot(439)
    plt.plot(range(1, len(loss_adam_7) + 1), loss_adam_7, label='train adam .7')
    plt.plot(range(1, len(val_loss_adam_7) + 1), val_loss_adam_7, label='valid adam .7')
    plt.legend(loc=loss_lengend, shadow=True, fontsize='x-large')
    plt.xlabel('Epochs')
    plt.ylabel('LogLoss');
    
    plt.subplot(4, 3, 10)
    plt.plot(range(1, len(loss_sgd_3) + 1), loss_sgd_3, label='train sgd .3')
    plt.plot(range(1, len(val_loss_sgd_3) + 1), val_loss_adam_3, label='valid sgd .3')
    plt.legend(loc=loss_lengend, shadow=True, fontsize='x-large')
    plt.xlabel('Epochs')
    plt.ylabel('LogLoss');
    
    plt.subplot(4, 3, 11)
    plt.plot(range(1, len(loss_sgd_5) + 1), loss_sgd_5, label='train sgd .5')
    plt.plot(range(1, len(val_loss_sgd_5) + 1), val_loss_sgd_5, label='valid sgd .5')
    plt.legend(loc=loss_lengend, shadow=True, fontsize='x-large')
    plt.xlabel('Epochs')
    plt.ylabel('LogLoss');
    
    plt.subplot(4, 3, 12)
    plt.plot(range(1, len(loss_sgd_7) + 1), loss_sgd_7, label='train sgd .7')
    plt.plot(range(1, len(val_loss_sgd_7) + 1), val_loss_sgd_7, label='valid sgd .7')
    plt.xticks(range(1, len(loss_adam_3) + 1));
    plt.legend(loc=loss_lengend, shadow=True, fontsize='x-large')
    plt.xlabel('Epochs')
    plt.ylabel('LogLoss');


## VGG16

In [ ]:
model_train('vgg16', 0.3, Adam(lr=0.001), 'small_vgg16_Adam_0.3')

In [ ]:
model_train('vgg16', 0.5, Adam(lr=0.001), 'small_vgg16_Adam_0.5')

In [ ]:
model_train('vgg16', 0.7, Adam(lr=0.001), 'small_vgg16_Adam_0.7')

In [ ]:
model_train('vgg16', 0.3, SGD(lr=0.001), 'small_vgg16_SGD_0.3')

In [ ]:
model_train('vgg16', 0.5, SGD(lr=0.001), 'small_vgg16_SGD_0.5')

In [ ]:
model_train('vgg16', 0.7, SGD(lr=0.001), 'small_vgg16_SGD_0.7')

## VGG19

In [ ]:
model_train('vgg19', 0.3, Adam(lr=0.001), 'small_vgg19_Adam_0.3')

In [ ]:
model_train('vgg19', 0.5, Adam(lr=0.001), 'small_vgg19_Adam_0.5')

In [ ]:
model_train('vgg19', 0.7, Adam(lr=0.001), 'small_vgg19_Adam_0.7')

In [ ]:
model_train('vgg19', 0.3, SGD(lr=0.001), 'small_vgg19_SGD_0.3')

In [ ]:
model_train('vgg19', 0.5, SGD(lr=0.001), 'small_vgg19_SGD_0.5')

In [ ]:
model_train('vgg19', 0.7, SGD(lr=0.001), 'small_vgg19_SGD_0.7')

## ResNet50

In [ ]:
model_train('resnet50', 0.3, Adam(lr=0.001), 'small_resnet50_Adam_0.3')

In [ ]:
model_train('resnet50', 0.5, Adam(lr=0.001), 'small_resnet50_Adam_0.5')

In [ ]:
model_train('resnet50', 0.7, Adam(lr=0.001), 'small_resnet50_Adam_0.7')

In [ ]:
model_train('resnet50', 0.3, SGD(lr=0.001), 'small_resnet50_SGD_0.3')

In [ ]:
model_train('resnet50', 0.5, SGD(lr=0.001), 'small_resnet50_SGD_0.5')

In [ ]:
model_train('resnet50', 0.7, SGD(lr=0.001), 'small_resnet50_SGD_0.7')

## InceptionV3

In [ ]:
model_train('inceptionV3', 0.3, Adam(lr=0.001), 'small_inceptionV3_Adam_0.3')

In [ ]:
model_train('inceptionV3', 0.5, Adam(lr=0.001), 'small_inceptionV3_Adam_0.5')

In [ ]:
model_train('inceptionV3', 0.7, Adam(lr=0.001), 'small_inceptionV3_Adam_0.7')

In [ ]:
model_train('inceptionV3', 0.3, SGD(lr=0.001), 'small_inceptionV3_SGD_0.3')

In [ ]:
model_train('inceptionV3', 0.5, SGD(lr=0.001), 'small_inceptionV3_SGD_0.5')

In [ ]:
model_train('inceptionV3', 0.7, SGD(lr=0.001), 'small_inceptionV3_SGD_0.7')

## Xception

In [ ]:
model_train('xception', 0.3, Adam(lr=0.001), 'small_xception_Adam_0.3')

In [ ]:
model_train('xception', 0.5, Adam(lr=0.001), 'small_xception_Adam_0.5')

In [ ]:
model_train('xception', 0.5, Adam(lr=0.001), 'small_xception_Adam_0.5')

In [ ]:
model_train('xception', 0.3, SGD(lr=0.001), 'small_xception_SGD_0.3')

In [ ]:
model_train('xception', 0.5, SGD(lr=0.001), 'small_xception_SGD_0.5')

In [ ]:
model_train('xception', 0.7, SGD(lr=0.001), 'small_xception_SGD_0.7')